# ICS 434: DATA SCIENCE FUNDAMENTALS

## Hierarchical Indexes on DataFrames and Series

---


## Announcements

* Homework assignment 1
   * Due on **Tuesday, February 14, 11:59 PM**
   * Refer to the `Homework submission guidelines` announcement in Laulima for preparing and submitting your homework.
   

* No weekly engagement for this week.


* Use Slack channel for questions and discussion on topics relevant to the course material. 
  * Be sure to read the `Policy for using class slack workspace` announcement in Laulima.


## Slide Credits

Most of these slides were taken from Mahdi Belcaid.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from IPython.display import Image

In [2]:
# np.random.seed(12345)
# pd.options.display.max_rows = 20
# plt.rc('figure', figsize=(10, 6))

### Hierarchical Indexing
- We briefly discussed hierarchical indexing in the context of `groupby`
- Hierarchical indexing allows you to have multiple index levels on an axis
- Despite the name (indexing), it applies to both indexes and columns (i.e., rows or columns)
- Each index in a `MutiIndex` object is referred to as a level

In [3]:
prog_languages = pd.DataFrame({"Course": ["Python", "Rust", "Python", "Rust"], 
                               "Nb_participants": [10, 30, 27, 18]})
prog_languages

,Course,Nb_participants
0,Python,10
1,Rust,30
2,Python,27
3,Rust,18


In [4]:
x = prog_languages.groupby("Course")
p = x.get_group("Python")
r = x.get_group("Rust")

display(p, r)

,Course,Nb_participants
0,Python,10
2,Python,27


,Course,Nb_participants
1,Rust,30
3,Rust,18


In [5]:
prog_languages.groupby("Course").sum()

,Nb_participants
Course,
Python,37
Rust,48


In [6]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.287463
   2    0.365591
   3    0.566857
b  1   -1.220080
   3   -0.764460
c  1    1.920888
   2   -1.281129
d  2    1.305209
   3    0.664229
dtype: float64

In [7]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [8]:
data[('b', 1)]


-1.2200803070063218

In [9]:
data['b', 1]

-1.2200803070063218

In [10]:
data['b':'c']


b  1   -1.220080
   3   -0.764460
c  1    1.920888
   2   -1.281129
dtype: float64

In [11]:
data.loc[['b', 'd']]

b  1   -1.220080
   3   -0.764460
d  2    1.305209
   3    0.664229
dtype: float64

In [12]:
data

a  1    0.287463
   2    0.365591
   3    0.566857
b  1   -1.220080
   3   -0.764460
c  1    1.920888
   2   -1.281129
d  2    1.305209
   3    0.664229
dtype: float64

In [13]:
# selection from an "inner" level

data.loc[: , 2]

a    0.365591
c   -1.281129
d    1.305209
dtype: float64

In [14]:
df = pd.DataFrame({"A": np.random.randn(9), "B": np.random.randn(9)}, 
                  index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
df

A         B
a 1  0.385579 -1.569376
  2 -1.843412  0.027742
  3  0.912109 -0.411634
b 1 -0.988108  1.254630
  3  0.261137 -0.867713
c 1 -1.476167  0.008143
  2 -1.403877 -0.673938
d 2 -0.399434  1.847917
  3  0.210476  1.892284

In [15]:
pd.IndexSlice?

In [16]:
#E.g., data.loc[: , 2] worked on a Series but does not work on a DataFrame

df.loc(axis=0)[pd.IndexSlice[:, 2]]

,,A,B
a,2,-1.843412,0.027742
c,2,-1.403877,-0.673938
d,2,-0.399434,1.847917


### Stacked vs. Unstacked Data

* Hierarchical indexes represent how the data is often collected
 * Ex. if variables in a hospital are measured and recorded in different files

```python
    File 1
    patient_ABC  LDL 112
    patient_ABC  HDL 48
    patient_CCX  LDL 112
    patient_VDM  LDL 112
    patient_ABC  VO2 112
    patient_CCZ  RER 48
    ...
```


* This format is preferred  when not all the variables are measured in all the patients

  * I.e., using columns for the variables would result in a large number of empty cells
  


### Stacking and Unstacking a DataFrame

* You can convert back and forth between a DataFrame and a hierarchical index with `stack()` and `unstack()`

<img src="images/stacking_DF.png">

In [17]:
x = pd.DataFrame(np.random.randn(12).reshape((4, 3)), 
                 index=list("abcd"), columns=[1, 2, 3])
x.loc["b", 2] = np.nan
x.loc["c", 3] = np.nan
x.loc["d", 1] = np.nan
x

,1,2,3
a,-0.212452,0.584628,0.759534
b,-1.024472,NaN,-1.030664
c,0.665867,0.835715,NaN
d,NaN,-1.961852,-2.400428


In [18]:
x  = x.stack()
x

a  1   -0.212452
   2    0.584628
   3    0.759534
b  1   -1.024472
   3   -1.030664
c  1    0.665867
   2    0.835715
d  2   -1.961852
   3   -2.400428
dtype: float64

In [19]:
x = x.unstack()
x

,1,2,3
a,-0.212452,0.584628,0.759534
b,-1.024472,NaN,-1.030664
c,0.665867,0.835715,NaN
d,NaN,-1.961852,-2.400428


### Hierarchical Indexes on Rows and Columns


* In a `DataFrame`, both axes can have a hierarchical index
* Always remember the labels are stored under `index` for the rows and `columns` for the columns

<img src="images/index_columns.png">

In [20]:
data = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
data

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [21]:
# name hierarchical levels

data.index.names = ['key1', 'key2']
data.columns.names = ['state', 'color']
data

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [22]:
print(data.index.get_level_values("key1"))
print("=" * 20)
print(data.index.get_level_values("key2"))

Index(['a', 'a', 'b', 'b'], dtype='object', name='key1')
Int64Index([1, 2, 1, 2], dtype='int64', name='key2')


In [23]:
print(data.columns.get_level_values("state"))
print("=" * 20)
print(data.columns.get_level_values("color"))

Index(['Ohio', 'Ohio', 'Colorado'], dtype='object', name='state')
Index(['Green', 'Red', 'Green'], dtype='object', name='color')


- What type of variable would the following return (DataFrame, Series, np.array, ...)?

```python
    data['Ohio']`
```


- How about the following?

```python
    data['Ohio', 'Geeen']`
```

In [24]:
print(type(data['Ohio']))

data['Ohio']

<class 'pandas.core.frame.DataFrame'>


color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [25]:
print(type(data['Ohio', 'Green']))

data['Ohio', 'Green']

<class 'pandas.core.series.Series'>


key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int32

### Creating and Assigning a `MultiIndex` Object

* You can create a `MultiIndex` using one of the constructors
  * Ex. create from arrays
* As long as the dimensions are compatible, you can assign the `MultiIndex` to either the `index` or `column` attribute of a `DataFrame` or a `Series`


In [26]:
data = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
data.index.names = ["letter", "number"]
data

0   1   2
letter number           
a      1       0   1   2
       2       3   4   5
b      1       6   7   8
       2       9  10  11

In [27]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [28]:
data.columns = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])
data

state          Ohio     Colorado
color         Green Red    Green
letter number                   
a      1          0   1        2
       2          3   4        5
b      1          6   7        8
       2          9  10       11

### Reordering and Sorting Levels
- `swaplevel` method takes two level numbers or names and returns a new object with the levels interchanged
- As with simple indexes, you can sort a single or a combination of indexes of the `MultiIndex` object

In [29]:
data = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
data.index.names = ['key1', 'key2']
data

0   1   2
key1 key2           
a    1     0   1   2
     2     3   4   5
b    1     6   7   8
     2     9  10  11

In [30]:
data.swaplevel('key1', 'key2')

,,0,1,2
key2,key1,,,
1,a,0,1,2
2,a,3,4,5
1,b,6,7,8
2,b,9,10,11


In [31]:
data.sort_index(level=1, ascending=False)

,,0,1,2
key1,key2,,,
b,2,9,10,11
a,2,3,4,5
b,1,6,7,8
a,1,0,1,2


In [32]:
# You can also pass the level's index number, instead of name
data.swaplevel(0, 1).sort_index(level=0, ascending=False)

0   1   2
key2 key1           
2    b     9  10  11
     a     3   4   5
1    b     6   7   8
     a     0   1   2

In [33]:
data.columns = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])
data

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [34]:
data.swaplevel(0, 1, 0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [35]:
data.swaplevel(0, 1, 1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

In [36]:
data.swaplevel('state', 'color', axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

### Summary Statistics by Level

* The summary statistics that we've seen before can also be applied by `level`
* When you apply a function on the level, the remaining levels get `"squashed"`
* While you will see such code, this approach is no longer recommended and won't be supported in future versions
  * Use `groupby` instead


In [37]:
data

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [38]:
data.sum(level="state", axis=1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_5616\1335975597.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  data.sum(level="state", axis=1)


state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

In [39]:
data.groupby(level='state', axis=1).sum()

state      Colorado  Ohio
key1 key2                
a    1            2     1
     2            5     7
b    1            8    13
     2           11    19

### Setting and Resetting the Index
* It's common to read in a DataFrame file and use one or more of its columns as the index
  * Setting one or more columns as the index is done using `set_index()`
    * By default, the columns are removed from the `DataFrame`, but you can choose to leave them in 

  * `reset_index()` does the opposite; moves the index as a new column of the `DataFrame`
    * Creates a new index from an `RangeIndex()`

In [40]:
data = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})
data

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [41]:
data2 = data.set_index('b')
data2

,a,c,d
b,,,
7,0,one,0
6,1,one,1
5,2,one,2
4,3,two,0
3,4,two,1
2,5,two,2
1,6,two,3


In [42]:
data3 = data.set_index(['c', 'd'])
data3

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [43]:
data.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [44]:
display(data2)

data4 = data2.reset_index()
data4

,a,c,d
b,,,
7,0,one,0
6,1,one,1
5,2,one,2
4,3,two,0
3,4,two,1
2,5,two,2
1,6,two,3


,b,a,c,d
0,7,0,one,0
1,6,1,one,1
2,5,2,one,2
3,4,3,two,0
4,3,4,two,1
5,2,5,two,2
6,1,6,two,3


In [45]:
data4.index

RangeIndex(start=0, stop=7, step=1)

In [46]:
display(data3)
data3.reset_index((1,))

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

,d,a,b
c,,,
one,0,0,7
one,1,1,6
one,2,2,5
two,0,3,4
two,1,4,3
two,2,5,2
two,3,6,1


In [47]:
data3.reset_index((1,), drop=True)

,a,b
c,,
one,0,7
one,1,6
one,2,5
two,3,4
two,4,3
two,5,2
two,6,1
